# 第8章: ニューラルネット

第7章で取り組んだポジネガ分類を題材として、ニューラルネットワークで分類モデルを実装する。なお、この章ではPyTorchやTensorFlow、JAXなどの深層学習フレームワークを活用せよ。

## 70. 単語埋め込みの読み込み

事前学習済み単語埋め込みを活用し、$|V| \times d_\rm{emb}$ の単語埋め込み行列$\pmb{E}$を作成せよ。ここで、$|V|$は単語埋め込みの語彙数、$d_\rm{emb}$は単語埋め込みの次元数である。ただし、単語埋め込み行列の先頭の行ベクトル$\pmb{E}_{0,:}$は、将来的にパディング（`<PAD>`）トークンの埋め込みベクトルとして用いたいので、ゼロベクトルとして予約せよ。ゆえに、$\pmb{E}$の2行目以降に事前学習済み単語埋め込みを読み込むことになる。

もし、Google Newsデータセットの[学習済み単語ベクトル](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing)（300万単語・フレーズ、300次元）を全て読み込んだ場合、$|V|=3000001, d_\rm{emb}=300$になるはずである（ただ、300万単語の中には、殆ど用いられない稀な単語も含まれるので、語彙を削減した方がメモリの節約になる）。

また、単語埋め込み行列の構築と同時に、単語埋め込み行列の各行のインデックス番号（トークンID）と、単語（トークン）への双方向の対応付けを保持せよ。

In [62]:
!pip install gensim

In [63]:
!pip install numpy

In [1]:
import gensim.downloader as api

In [2]:
import gensim.downloader as api

# Google Newsの学習済み単語ベクトルをダウンロード・ロード
word2vec = api.load('word2vec-google-news-300')


[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [3]:
word2id = {"<PAD>": 0}
id2word = {0: "<PAD>"} #<PAD>を予約しておく

for i, word in enumerate(word2vec.index_to_key):  # index_to_key は語彙のリスト
    word2id[word] = i + 1  # 0は<PAD>だから+1
    id2word[i + 1] = word

In [4]:
import torch
import torch.nn as nn
import numpy as np

In [5]:
vocab_size = len(word2vec) + 1  # <PAD>のために+1
emb_dim = word2vec.vector_size

embedding_tensor = torch.zeros((vocab_size, emb_dim), dtype=torch.float32)

embedding_tensor[1:] = torch.from_numpy(word2vec.vectors)

## 71. データセットの読み込み

[General Language Understanding Evaluation (GLUE)](https://gluebenchmark.com/) ベンチマークで配布されている[Stanford Sentiment Treebank (SST)](https://dl.fbaipublicfiles.com/glue/data/SST-2.zip) をダウンロードし、訓練セット（train.tsv）と開発セット（dev.tsv）のテキストと極性ラベルと読み込み、全てのテキストをトークンID列に変換せよ。このとき、単語埋め込みの語彙でカバーされていない単語は無視し、トークン列に含めないことにせよ。また、テキストの全トークンが単語埋め込みの語彙に含まれておらず、空のトークン列となってしまう事例は、訓練セットおよび開発セットから削除せよ（このため、第7章の実験で得られた正解率と比較できなくなることに注意せよ）。

事例の表現方法は任意でよいが、例えば"contains no wit , only labored gags"がネガティブに分類される事例は、次のような辞書オブジェクトで表現すればよい。

```
{'text': 'contains no wit , only labored gags',
 'label': tensor([0.]),
 'input_ids': tensor([ 3475,    87, 15888,    90, 27695, 42637])}
```

この例では、`text`はテキスト、`label`は分類ラベル（ポジティブなら`tensor([1.])`、ネガティブなら`tensor([0.])`）、`input_ids`はテキストのトークン列をID列で表現している。

In [7]:
!wget https://dl.fbaipublicfiles.com/glue/data/SST-2.zip
!unzip SST-2.zip

--2025-05-18 15:21:39--  https://dl.fbaipublicfiles.com/glue/data/SST-2.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.238.176.115, 18.238.176.44, 18.238.176.19, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.238.176.115|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7439277 (7.1M) [application/zip]
Saving to: ‘SST-2.zip.1’

SST-2.zip.1         100%[===================>]   7.09M  --.-KB/s    in 0.1s    

2025-05-18 15:21:40 (53.3 MB/s) - ‘SST-2.zip.1’ saved [7439277/7439277]

Archive:  SST-2.zip
replace SST-2/dev.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: SST-2/dev.tsv           
replace SST-2/original/README.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: SST-2/original/README.txt  
replace SST-2/original/SOStr.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: SST-2/original/SOStr.txt  
replace SST-2/original/STree.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: SST-2/origi

In [8]:
import pandas as pd

# データの読み込み
train_df = pd.read_csv("SST-2/train.tsv", sep='\t', )
dev_df = pd.read_csv("SST-2/dev.tsv", sep='\t', )


Train: Positive = 37569 , Negative = 29780
Valid: Positive = 444 , Negative = 428


,sentence,label
0,hide new secretions from the parental units,0
1,"contains no wit , only labored gags",0
2,that loves its characters and communicates som...,1
3,remains utterly satisfied to remain the same t...,0
4,on the worst revenge-of-the-nerds clichés the ...,0


In [9]:
def text_to_ids(text, word2id):
    ids = []
    for word in text.split():
        if word in word2id:
            ids.append(word2id[word])
    return ids

In [10]:
def pre_processing(df, word2id):
  data = []
  for index, row in df.iterrows():
    data_unit = {}
    ids = text_to_ids(row["sentence"], word2id)
    if len(ids) > 0:
      data_unit["text"] = row["sentence"]
      data_unit["label"] = torch.tensor([row["label"]], dtype=torch.float32)
      data_unit["input_ids"] = torch.tensor(ids, dtype=torch.long)
      data.append(data_unit)
  return data


In [11]:
train_data = pre_processing(train_df, word2id)
dev_data = pre_processing(dev_df, word2id)

In [12]:
train_data[:5]

[{'text': 'hide new secretions from the parental units ',
  'label': tensor([0.]),
  'input_ids': tensor([  5785,     66, 113845,     18,     12,  15095,   1594])},
 {'text': 'contains no wit , only labored gags ',
  'label': tensor([0.]),
  'input_ids': tensor([ 3475,    87, 15888,    90, 27695, 42637])},
 {'text': 'that loves its characters and communicates something rather beautiful about human nature ',
  'label': tensor([1.]),
  'input_ids': tensor([    4,  5053,    45,  3305, 31647,   348,   904,  2815,    47,  1276,
           1964])},
 {'text': 'remains utterly satisfied to remain the same throughout ',
  'label': tensor([0.]),
  'input_ids': tensor([  987, 14528,  4941,   873,    12,   208,   898])},
 {'text': 'on the worst revenge-of-the-nerds clichés the filmmakers could dredge up ',
  'label': tensor([0.]),
  'input_ids': tensor([    6,    12,  1445, 43789,    12, 10946,    76, 41349,    42])}]

## 72. Bag of wordsモデルの構築

単語埋め込みの平均ベクトルでテキストの特徴ベクトルを表現し、重みベクトルとの内積でポジティブ及びネガティブを分類するニューラルネットワーク（ロジスティック回帰モデル）を設計せよ。

In [13]:
def input_ids2mean_feature(input_ids, embedding_tensor):
    selected_vectors = embedding_tensor[input_ids]
    mean_vector = torch.mean(selected_vectors, dim=0)

    return mean_vector

In [14]:
ex_mean = input_ids2mean_feature(train_data[0]["input_ids"], embedding_tensor)
ex_mean.shape

torch.Size([300])

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LogisticRegression(nn.Module): #ロジスティック回帰の設計
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        h1 = self.linear(x)
        h2 = torch.sigmoid(h1)
        return h2

## 73. モデルの学習

問題72で設計したモデルの重みベクトルを訓練セット上で学習せよ。ただし、学習中は単語埋め込み行列の値を固定せよ（単語埋め込み行列のファインチューニングは行わない）。また、学習時に損失値を表示するなど、学習の進捗状況をモニタリングできるようにせよ。

In [16]:
model = LogisticRegression(emb_dim, 1)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for data_unit in train_data:
        # 特徴ベクトルの計算
        feature_vector = input_ids2mean_feature(data_unit["input_ids"], embedding_tensor)

        #modelが予測する
        output = model(feature_vector)

        loss = criterion(output, data_unit["label"])

        # バックプロパゲーションとパラメータ更新
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_data):.4f}')


Epoch 1/10, Loss: 0.4024
Epoch 2/10, Loss: 0.3725
Epoch 3/10, Loss: 0.3701
Epoch 4/10, Loss: 0.3692
Epoch 5/10, Loss: 0.3688
Epoch 6/10, Loss: 0.3686
Epoch 7/10, Loss: 0.3685
Epoch 8/10, Loss: 0.3684
Epoch 9/10, Loss: 0.3684
Epoch 10/10, Loss: 0.3683


## 74. モデルの評価

問題73で学習したモデルの開発セットにおける正解率を求めよ。

In [39]:
def evaluate_accuracy(model, dev_data, embedding_tensor):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for data_unit in dev_data:
            feature_vector = input_ids2mean_feature(data_unit["input_ids"], embedding_tensor)

            output = model(feature_vector)

            pred = (output >= 0.5).float()

            correct += (pred == data_unit["label"]).sum().item()
            total += 1

    accuracy = correct / total
    return accuracy



In [40]:
accuracy = evaluate_accuracy(model, dev_data, embedding_tensor)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 79.82%


## 75. パディング

複数の事例が与えられたとき、これらをまとめて一つのテンソル・オブジェクトで表現する関数`collate`を実装せよ。与えられた複数の事例のトークン列の長さが異なるときは、トークン列の長さが最も長いものに揃え、0番のトークンIDでパディングをせよ。さらに、トークン列の長さが長いものから順に、事例を並び替えよ。

例えば、訓練データセットの冒頭の4事例が次のように表されているとき、

```
[{'text': 'hide new secretions from the parental units',
  'label': tensor([0.]),
  'input_ids': tensor([  5785,     66, 113845,     18,     12,  15095,   1594])},
 {'text': 'contains no wit , only labored gags',
  'label': tensor([0.]),
  'input_ids': tensor([ 3475,    87, 15888,    90, 27695, 42637])},
 {'text': 'that loves its characters and communicates something rather beautiful about human nature',
  'label': tensor([1.]),
  'input_ids': tensor([    4,  5053,    45,  3305, 31647,   348,   904,  2815,    47,  1276,  1964])},
 {'text': 'remains utterly satisfied to remain the same throughout',
  'label': tensor([0.]),
  'input_ids': tensor([  987, 14528,  4941,   873,    12,   208,   898])}]
```

`collate`関数を通した結果は以下のようになることが想定される。

```
{'input_ids': tensor([
    [     4,   5053,     45,   3305,  31647,    348,    904,   2815,     47,   1276,   1964],
    [  5785,     66, 113845,     18,     12,  15095,   1594,      0,      0,      0,      0],
    [   987,  14528,   4941,    873,     12,    208,    898,      0,      0,      0,      0],
    [  3475,     87,  15888,     90,  27695,  42637,      0,      0,      0,      0,      0]]),
 'label': tensor([
    [1.],
    [0.],
    [0.],
    [0.]])}
```


In [28]:
def collate(batch):
    # トークン長が長い順にソート
    batch = sorted(batch, key=lambda x: len(x["input_ids"]), reverse=True)

    max_len = len(batch[0]["input_ids"])  #sortしたあとだから，0番目がmax

    padded_input_ids = []
    labels = []

    for data_unit in batch:
        input_ids = data_unit["input_ids"]
        label = data_unit["label"]

        # パディングする数
        pad_len = max_len - len(input_ids)
        padded = torch.cat([input_ids, torch.zeros(pad_len, dtype=torch.long)])  #pad_lenの数だけ下のinput_idsに0を連結する

        padded_input_ids.append(padded) #この時はshape: (max_len)のテンソルのリスト
        labels.append(label)

    # バッチテンソルに変換
    padded_input_ids = torch.stack(padded_input_ids)  # shape: (batch_size, max_len)　torch.stack使えばリストの中にテンソルが入っている状態のものを一つのテンソルにー
    labels = torch.stack(labels)  # shape: (batch_size, 1)

    return {
        "input_ids": padded_input_ids,
        "label": labels
    }


In [29]:
ex_padding = collate(train_data[:5])
ex_padding

{'input_ids': tensor([[     4,   5053,     45,   3305,  31647,    348,    904,   2815,     47,
            1276,   1964],
         [     6,     12,   1445,  43789,     12,  10946,     76,  41349,     42,
               0,      0],
         [  5785,     66, 113845,     18,     12,  15095,   1594,      0,      0,
               0,      0],
         [   987,  14528,   4941,    873,     12,    208,    898,      0,      0,
               0,      0],
         [  3475,     87,  15888,     90,  27695,  42637,      0,      0,      0,
               0,      0]]),
 'label': tensor([[1.],
         [0.],
         [0.],
         [0.],
         [0.]])}

## 76. ミニバッチ学習

問題75のパディングの処理を活用して、ミニバッチでモデルを学習せよ。また、学習したモデルの開発セットにおける正解率を求めよ。

In [30]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
  def __init__(self, data):
    self.data = data

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data[idx]

In [31]:
train_dataset = MyDataset(train_data)
train_dataset.__getitem__(0) #確認してみた

{'text': 'hide new secretions from the parental units ',
 'label': tensor([0.]),
 'input_ids': tensor([  5785,     66, 113845,     18,     12,  15095,   1594])}

In [32]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate)

In [33]:
i = 0
for data in train_dataloader:
    print(data)
    i = i+1
    if i > 3:
      break #確認してみた


{'input_ids': tensor([[     3,     53,     45,   1023,    451,    649,    112,    677,    184,
         198991,    179,     42,    121,    139,  10012,     30,  22480],
        [     5,   2441,      9,  90055,   3463, 100498,   3305,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0],
        [    77,  55120, 198991,    143,     16,    102,     40,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0],
        [     2,   1192,  36265,   5908,   6161,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0],
        [  2377,   3036,      6,     45,    413,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0],
        [    12,   3913,      5,  27154,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0],
        [ 51846,   6227,   7014,      0,      0,      0,

In [42]:
model_with_batch_learning = LogisticRegression(emb_dim, 1)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model_with_batch_learning.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    model_with_batch_learning.train()
    total_loss = 0
    for batch in train_dataloader:

      input_ids_batch = batch['input_ids'] #shape: (B, L)
      label_batch = batch['label'].float().squeeze(1) #shape: (B,)

      # 特徴量の平均ベクトルをバッチ単位で計算
        # embedding_tensor: shape (Vocab_size, emb_dim)
        # input_ids_batch: shape (B, L)
      embedded = embedding_tensor[input_ids_batch]  # shape: (B, L, D)
      mean_vectors = embedded.mean(dim=1)           # shape: (B, D)

      outputs = model_with_batch_learning(mean_vectors).squeeze(1)      # shape: (B,)

      loss = criterion(outputs, label_batch)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      total_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_dataloader):.4f}")



Epoch 1/10, Loss: 0.5430
Epoch 2/10, Loss: 0.4486
Epoch 3/10, Loss: 0.4240
Epoch 4/10, Loss: 0.4132
Epoch 5/10, Loss: 0.4059
Epoch 6/10, Loss: 0.4013
Epoch 7/10, Loss: 0.3973
Epoch 8/10, Loss: 0.3960
Epoch 9/10, Loss: 0.3950
Epoch 10/10, Loss: 0.3945


In [43]:
accuracy = evaluate_accuracy(model_with_batch_learning, dev_data, embedding_tensor)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 80.28%


## 77. GPU上での学習

問題76のモデル学習をGPU上で実行せよ。また、学習したモデルの開発セットにおける正解率を求めよ。

In [44]:
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model_with_batch_learning = LogisticRegression(emb_dim, 1).to(device)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model_with_batch_learning.parameters(), lr=0.001)

embedding_tensor = embedding_tensor.to(device)

num_epochs = 10

for epoch in range(num_epochs):
    model_with_batch_learning.train()
    total_loss = 0
    for batch in train_dataloader:

        input_ids_batch = batch['input_ids'].to(device)        # shape: (B, L)
        label_batch = batch['label'].float().squeeze(1).to(device)  # shape: (B,)

        # embedding_tensor: shape (Vocab_size, emb_dim)
        # input_ids_batch: shape (B, L)
        embedded = embedding_tensor[input_ids_batch]  # shape: (B, L, D)
        mean_vectors = embedded.mean(dim=1)           # shape: (B, D)

        outputs = model_with_batch_learning(mean_vectors).squeeze(1)  # shape: (B,)

        loss = criterion(outputs, label_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_dataloader):.4f}")


Using device: cpu


KeyboardInterrupt: 

## 78. 単語埋め込みのファインチューニング

問題77の学習において、単語埋め込みのパラメータも同時に更新するファインチューニングを導入せよ。また、学習したモデルの開発セットにおける正解率を求めよ。

In [ ]:
embedding_layer = nn.Embedding.from_pretrained(embedding_tensor, freeze=False).to(device)
model_with_batch_learning = LogisticRegression(emb_dim, 1).to(device)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(
    list(model_with_batch_learning.parameters()) + list(embedding_layer.parameters()),
    lr=0.001
)

num_epochs = 10

for epoch in range(num_epochs):
    model_with_batch_learning.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids_batch = batch['input_ids'].to(device)
        label_batch = batch['label'].float().squeeze(1).to(device)

        embedded = embedding_layer(input_ids_batch)  # shape: (B, L, D)
        mean_vectors = embedded.mean(dim=1)          # shape: (B, D)

        outputs = model_with_batch_learning(mean_vectors).squeeze(1)

        loss = criterion(outputs, label_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_dataloader):.4f}")


## 79. アーキテクチャの変更

ニューラルネットワークのアーキテクチャを自由に変更し、モデルを学習せよ。また、学習したモデルの開発セットにおける正解率を求めよ。例えば、テキストの特徴ベクトル（単語埋め込みの平均ベクトル）に対して多層のニューラルネットワークを通したり、畳み込みニューラルネットワーク（CNN; Convolutional Neural Network）や再帰型ニューラルネットワーク（RNN; Recurrent Neural Network）などのモデルの学習に挑戦するとよい。